# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load agemoea results for all topos

In [2]:
with open('res_bc_Cogent_agemoea.pkl','rb') as f_Cogent:
    res_1_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_agemoea.pkl','rb') as f_UsCarrier:
    res_1_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_agemoea.pkl','rb') as f_HiberniaGlobal:
    res_1_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_agemoea.pkl','rb') as f_Colt:
    res_1_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_agemoea.pkl','rb') as f_Funet:
    res_1_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_agemoea.pkl','rb') as f_Abvt:
    res_1_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_agemoea.pkl','rb') as f_Intellifiber:
    res_1_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_agemoea.pkl','rb') as f_TataNld:
    res_1_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_agemoea.pkl','rb') as f_Internode:
    res_1_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_agemoea.pkl','rb') as f_Missouri:
    res_1_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_agemoea.pkl','rb') as f_Ion:
    res_1_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_agemoea.pkl','rb') as f_Palmetto:
    res_1_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:

# agemoea
F1_Cogent=res_1_Cogent.F
F1_UsCarrier=res_1_UsCarrier.F
F1_HiberniaGlobal=res_1_HiberniaGlobal.F
F1_Colt=res_1_Colt.F
F1_Funet=res_1_Funet.F
F1_Abvt=res_1_Abvt.F
F1_Intellifiber=res_1_Intellifiber.F
F1_TataNld=res_1_TataNld.F
F1_Internode=res_1_Internode.F
F1_Missouri=res_1_Missouri.F
F1_Ion=res_1_Ion.F
F1_Palmetto=res_1_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F1_Cogent = []
hist_F1_UsCarrier = []
hist_F1_HiberniaGlobal = []
hist_F1_Colt = []
hist_F1_Funet = []
hist_F1_Abvt = []
hist_F1_Intellifiber = []
hist_F1_TataNld = []
hist_F1_Internode = []
hist_F1_Missouri = []
hist_F1_Ion = []
hist_F1_Palmetto = []


for algo1_Cogent in res_1_Cogent.history:
    opt1_Cogent = algo1_Cogent.opt
    feas1_Cogent = np.where(opt1_Cogent.get("feasible"))[0]
    hist_F1_Cogent.append(opt1_Cogent.get("F")[feas1_Cogent])
for algo1_UsCarrier in res_1_UsCarrier.history:
    opt1_UsCarrier = algo1_UsCarrier.opt
    feas1_UsCarrier = np.where(opt1_UsCarrier.get("feasible"))[0]
    hist_F1_UsCarrier.append(opt1_UsCarrier.get("F")[feas1_UsCarrier])
for algo1_HiberniaGlobal in res_1_HiberniaGlobal.history:
    opt1_HiberniaGlobal = algo1_HiberniaGlobal.opt
    feas1_HiberniaGlobal = np.where(opt1_HiberniaGlobal.get("feasible"))[0]
    hist_F1_HiberniaGlobal.append(opt1_HiberniaGlobal.get("F")[feas1_HiberniaGlobal])
for algo1_Colt in res_1_Colt.history:
    opt1_Colt = algo1_Colt.opt
    feas1_Colt = np.where(opt1_Colt.get("feasible"))[0]
    hist_F1_Colt.append(opt1_Colt.get("F")[feas1_Colt])
for algo1_Funet in res_1_Funet.history:
    opt1_Funet = algo1_Funet.opt
    feas1_Funet = np.where(opt1_Funet.get("feasible"))[0]
    hist_F1_Funet.append(opt1_Funet.get("F")[feas1_Funet])
for algo1_Abvt in res_1_Abvt.history:
    opt1_Abvt = algo1_Abvt.opt
    feas1_Abvt = np.where(opt1_Abvt.get("feasible"))[0]
    hist_F1_Abvt.append(opt1_Abvt.get("F")[feas1_Abvt])
for algo1_Intellifiber in res_1_Intellifiber.history:
    opt1_Intellifiber = algo1_Intellifiber.opt
    feas1_Intellifiber = np.where(opt1_Intellifiber.get("feasible"))[0]
    hist_F1_Intellifiber.append(opt1_Intellifiber.get("F")[feas1_Intellifiber])
for algo1_TataNld in res_1_TataNld.history:
    opt1_TataNld = algo1_TataNld.opt
    feas1_TataNld = np.where(opt1_TataNld.get("feasible"))[0]
    hist_F1_TataNld.append(opt1_TataNld.get("F")[feas1_TataNld])
for algo1_Internode in res_1_Internode.history:
    opt1_Internode = algo1_Internode.opt
    feas1_Internode= np.where(opt1_Internode.get("feasible"))[0]
    hist_F1_Internode.append(opt1_Internode.get("F")[feas1_Internode])
for algo1_Missouri in res_1_Missouri.history:
    opt1_Missouri = algo1_Missouri.opt
    feas1_Missouri = np.where(opt1_Missouri.get("feasible"))[0]
    hist_F1_Missouri.append(opt1_Missouri.get("F")[feas1_Missouri])
for algo1_Ion in res_1_Ion.history:
    opt1_Ion = algo1_Ion.opt
    feas1_Ion = np.where(opt1_Ion.get("feasible"))[0]
    hist_F1_Ion.append(opt1_Ion.get("F")[feas1_Ion])
for algo1_Palmetto in res_1_Palmetto.history:
    opt1_Palmetto = algo1_Palmetto.opt
    feas1_Palmetto = np.where(opt1_Palmetto.get("feasible"))[0]
    hist_F1_Palmetto.append(opt1_Palmetto.get("F")[feas1_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for agemoea algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_1_Cogent = [metric.do(_F1_Cogent) for _F1_Cogent in hist_F1_Cogent]
hv_1_UsCarrier = [metric.do(_F1_UsCarrier) for _F1_UsCarrier in hist_F1_UsCarrier]
hv_1_HiberniaGlobal = [metric.do(_F1_HiberniaGlobal) for _F1_HiberniaGlobal in hist_F1_HiberniaGlobal]
hv_1_Colt = [metric.do(_F1_Colt) for _F1_Colt in hist_F1_Colt]
hv_1_Funet = [metric.do(_F1_Funet) for _F1_Funet in hist_F1_Funet]
hv_1_Abvt = [metric.do(_F1_Abvt) for _F1_Abvt in hist_F1_Abvt]
hv_1_Intellifiber = [metric.do(_F1_Intellifiber) for _F1_Intellifiber in hist_F1_Intellifiber]
hv_1_TataNld = [metric.do(_F1_TataNld) for _F1_TataNld in hist_F1_TataNld]
hv_1_Internode = [metric.do(_F1_Internode) for _F1_Internode in hist_F1_Internode]
hv_1_Missouri = [metric.do(_F1_Missouri) for _F1_Missouri in hist_F1_Missouri]
hv_1_Ion = [metric.do(_F1_Ion) for _F1_Ion in hist_F1_Ion]
hv_1_Palmetto = [metric.do(_F1_Palmetto) for _F1_Palmetto in hist_F1_Palmetto]

# agemoea
hv_1 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_1_Cogent, hv_1_UsCarrier, hv_1_HiberniaGlobal, hv_1_Colt, hv_1_Funet, hv_1_Abvt, hv_1_Intellifiber, hv_1_TataNld, hv_1_Internode, hv_1_Missouri, hv_1_Ion, hv_1_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_1_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_1_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_1_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_1_Colt)
print('The hypervolumn for Funet:')
print(hv_1_Funet)
print('The hypervolumn for Abvt:')
print(hv_1_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_1_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_1_TataNld)
print('The hypervolumn for Internode:')
print(hv_1_Internode)
print('The hypervolumn for Missouri:')
print(hv_1_Missouri)
print('The hypervolumn for Ion:')
print(hv_1_Ion)
print('The hypervolumn for Palmetto:')
print(hv_1_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_1_Cogent)+1)), hv_1_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_1_UsCarrier)+1)), hv_1_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_1_HiberniaGlobal)+1)), hv_1_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_1_Colt)+1)), hv_1_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_1_Funet)+1)), hv_1_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_1_Abvt)+1)), hv_1_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_1_Intellifiber)+1)), hv_1_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_1_TataNld)+1)), hv_1_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_1_Internode)+1)), hv_1_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_1_Missouri)+1)), hv_1_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_1_Ion)+1)), hv_1_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_1_Palmetto)+1)), hv_1_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for agemoea:')
print(hv_1)

3. Draw mean hypervolume for agemoea

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()